# Задача распознавания говорящего по голосу

## I. Постановка задачи

На основании имеющихся данных мы хотим научиться определять говорящего. **Data-set** представляет собой набор, состоящий из
1. Trainig corpus. Голосовые высказывания спикеров (несколько записей по каждому спикеру).
2. Test corpus. Другие записи тех же спикеров.

Все аудиофайлы имеют длительность 10 sec и сэмплируются на частоте 16000 Hz.

## II. Введение

Речевой сигнал представляет собой последовательность чисел, которые определяют амплитуду говорящего. Вся концепция распознования речи базируется на трёх основных принципах/инструментах:

### 1. Framing

Поскольку речь не является стационарным сигналом (стационарностью называется свойство процесса сохранять свои характеристики с течением времени), её частотное содержание постоянно изменяется во времени. Чтобы выполнить хотя бы какой-нибудь анализ сигнала на коротких временных интервалах (*Short Term Fourier Transformation*), нам надо иметь возможность рассмотреть сигнал как стационарный. Чтобы достичь этой стационарности, речевой сигнал делится на ***фреймы*** длинной 20-30 ms. На такой длине можно сделать предположение о том, что форма волны не изменяется или изменяется совсем незначительно.

![frame-segmentation](presentation/Segmentation-of-speech-signals-frame-by-frame.png)

### 2. Windowing

Извлечение фреймов из речевого сигнала может привести к разрывам в целых точках из-за нецелого числа периодов в извлечённом сигнале, что в свою очередь может привести к ошибочному представлению частоты (говорят, что случается *спектральная утечка* ~ *spectral leakage*). Это можно предотвратить умножением фрейма на некоторую ***оконную функцию***. Амплитуда оконной функции падает до нуля на концах фрейма, что естественным образом минимизирует амплитуду разрыва.

На картинках ниже приведена иллюстррация фрейма до и после умножения на оконную функцию.

![non-integer](presentation/noninteger1.png) ![windowing](presentation/window2.png)

### 3. Overlapping

Из-за windowing'а возможна ситуация, когда в результате "сужения" мы теряем часть информации о сигнале на концах фрейма. Чтобы нивелировать этот возможный деффект необходимо сделать ***перекрытие*** фреймов. Суть его в следующем: пусть фрейм *s* имеет длину 20-30 ms, возьмём фрейм *s+1*, который также имеет длину 20-30 ms и "наложим" его частично на фрейм *i*, длина перекрытия обычно равна 10-15 ms.

Приведём поясняющую иллюстрацию.

![overlapping](presentation/mfcc_audioframes.png)

## III. Gaussian Mixture Model (GMM)

***Gaussian Mixture Model (GMM)*** – вероятностная модель кластеризации для представления присутствия под-популяции в охватывающей популяции. Идея обучения GMM – приблизить распределение вероятности класса линейной комбинацией $k$ гауссовых распределений, которые также называются компонентами GMM.

Вероятность точек данных (векторов признака) для модели задаётся следующим образом

$\mathbf{P} (\mathbf{x} \vert \lambda) = \sum_{k = 1}^{K} \omega_k f_{\mathbf{X}_k}(\mathbf{x})$, где

$f_{\mathbf{X}}(\mathbf{x}) = \frac{1}{(2\pi )^{n/2} \vert \Sigma \vert^{1/2}} e^{-\frac{1}{2}(\mathbf{x} - \mathbf{\mu})^{\top} \Sigma^{-1} (\mathbf{x} - \mathbf{\mu})},\; \mathbf{x} \in \mathbb{R}^n$, 

где $\vert \Sigma\vert$ — определитель матрицы $\Sigma$, а $\Sigma^{-1}$ — матрица, обратная к $\Sigma$.


Training data $X_i$ класса $\lambda$ используется для оценки значений всех параметров.

Вначале определяется $k$ классв в данных $K$-средним алгоритмом и с весами $\omega = 1/k$ для каждого кластера. Затем $k$ гауссовых распределений фитят $k$ кластеров, все параметры при этом обновляются итеративно.

## IV. Демонстрация решения

### Установка и импорт необходимых пакетов

In [ ]:
%pip install numba==0.48.0
%pip install librosa
%pip install cffi==1.14.0
%pip show numba

In [42]:
import time
import os
from tqdm import tqdm

import numpy as np
import pandas as pd
import scipy

import librosa
from librosa import display
from IPython.display import Audio 

from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
import pickle

### Получение данных

Snippets -> Extract Zip file.py

### Генерация аудио-признаков

Создадим функцию, которая будет извлекать аудио-признаки

In [43]:
def audio_features_extraction(sample, sample_rate, n_fft):
    
    """ 
        sample - audio time series
        sample_rate - sampling rate of sample
        n_fft = frame size
    """
    
    # librosa.feature.mfcc - вычилсяет коэффициенты MFCCs.
    # MFCCs трансформируют значение сигнала в кепстр – один из видов гомоморфной обработки сигналов, 
    # функция обратного преобразования Фурье от логарифма спектра мощности сигнала. 
    # Основная задача: охарактеризовать фильтр и отделить исходную часть
    # (на примере с голосом человека – охарактеризовать вокальный тракт).
    mfcc = librosa.feature.mfcc(y=sample, 
                                n_fft=n_fft, # размер фрейма
                                window='hann',  # оконная функция (windowing)
                                hop_length=int(n_fft*0.5), # размер перекрытия фреймов (overlapping)
                                sr=sample_rate, 
                                n_mfcc=20)
    features = np.mean(mfcc, axis=1)
    
    # librosa.feature.zero_crossing находит нулевые переходы для сигнала.
    zero_crossings = sum(librosa.zero_crossings(sample, pad=False))
    features = np.append(zero_crossings, features)
    
    # librosa.feature.spectral_centroid вычисляет спектральный центроид.
    # Каждый фрейм амплитудной спектрограммы нормализуется и обрабатывается как распределение по частотным элементам,
    # из которого извлекается среднее значение (центроид) для каждого фрейма.
    spec_cent = librosa.feature.spectral_centroid(y=sample,n_fft=n_fft, hop_length=int(n_fft*0.5), window='hann', sr=sample_rate).mean()
    features = np.append(spec_cent, features)
    
    # librosa.feature.spectral_flatness вычисляет cпектральную плоскостность.
    # Спектральная плоскостность - количественная мера того, насколько звук похож на шум, а не на тон.
    spec_flat = librosa.feature.spectral_flatness(y=sample,n_fft=n_fft, hop_length=int(n_fft*0.5), window='hann').mean()
    features = np.append(spec_flat, features)
    
    # librosa.feature.spectral_bandwith вычисляет спектральную полосу пропускания p-ого порядка.
    spec_bw = librosa.feature.spectral_bandwidth(y=sample,n_fft=n_fft, hop_length=int(n_fft*0.5), window='hann', sr=sample_rate).mean()
    features = np.append(spec_bw, features)
    
    # librosa.feature.spectral_rolloff вычисляет roll-off частоту для каждого фрейма.
    # Roll-off частота определяется как центральная частота для интервала спектрограммы.
    rolloff = librosa.feature.spectral_rolloff(y=sample, n_fft=n_fft, hop_length=int(n_fft*0.5), window='hann', sr=sample_rate).mean()
    features = np.append(rolloff, features)
    
    return features

Читаем последовательно аудио-файлы и извлекаем аудио-признаки

In [44]:
start = time.time()

source   = "./webinar0813-yandex-gbc/data/development_set/"  
dest = "./webinar0813-yandex-gbc/speaker-models/"
train_file = "./webinar0813-yandex-gbc/data/development_set_enroll.txt"
file_paths = open(train_file, 'r')
 
n_fft = 1024

# Подготова датасетов для обучения моделей.
features = pd.DataFrame()
speakers = pd.DataFrame()

# Последовательное чтение аудио-файлов из тренировочного семпла.
for path in tqdm(file_paths, desc='Features extractions '):
    path = path.replace("\\","/").strip()
    speaker = path.split("-",1)[0]

    # librosa.load - загрузка аудио-файла.
    sample, sample_rate = librosa.load(source+path)
    
    # Извлечение аудио-признаков.
    features = features.append(pd.Series(audio_features_extraction(sample, sample_rate, n_fft)), ignore_index=True) 
    speakers = speakers.append({'speaker' : speaker}, ignore_index=True)
 
print('Execution time: ', round((time.time() - start),2))   

Features extractions : 170it [01:14,  2.27it/s]

Execution time:  75.0




/py-env/platform-env/lib/python3.7/site-packages/ml_platform/user_messages.py:13: UserWarning: 

The following variables cannot be serialized: features



Please note that these variables can be lost in the next working session

  warnings.warn(message)


Нормируем признаки

In [45]:
scaler = StandardScaler()
scaler.fit(features)
features_scaled = pd.DataFrame(scaler.transform(features))
pickle.dump(scaler, open('scaler','wb'))

/py-env/platform-env/lib/python3.7/site-packages/ml_platform/user_messages.py:13: UserWarning: 

The following variables cannot be serialized: features_scaled, features



Please note that these variables can be lost in the next working session

  warnings.warn(message)


### Обучение моделей

Обучаем модели и сохраняем в директорию `speaker-models/`

In [46]:
start = time.time()

features_smpl = pd.DataFrame()
count = 1

for i in range(speakers.shape[0]):
    speaker = speakers.iloc[i:i+1].values[0]
    if count == 1:
        speaker_prev = speaker     
    else :
        # Обучение модели GaussianMixture для каждого спикера
        if (speaker_prev != speaker) | (i == len(speakers)-1) :
            gmm = GaussianMixture(n_components = min(16, features_smpl.shape[0]), 
                                  max_iter = 200, covariance_type='diag', n_init = 3)
            gmm.fit(features_smpl)
            
            # Сохранение полученной модели в pickle
            pickle.dump(gmm, open((dest+speaker_prev)[0],'wb'))
            features_smpl = pd.DataFrame()
            count = 0
            speaker_prev = speaker
            
    # Сбор данных по одному спикеру.
    features_smpl  = features_smpl.append(features_scaled.iloc[i:i+1], ignore_index=True)
    count = count+1

print('Execution time: ', round((time.time() - start),2)) 

Execution time:  0.65


/py-env/platform-env/lib/python3.7/site-packages/ml_platform/user_messages.py:13: UserWarning: 

The following variables cannot be serialized: features_scaled, features_smpl



Please note that these variables can be lost in the next working session

  warnings.warn(message)


### Тестирование на тестовом семпле

In [47]:
start = time.time()

source   = "./webinar0813-yandex-gbc/data/development_set/"  
dest = "./webinar0813-yandex-gbc/speaker-models/"
test_file = "./webinar0813-yandex-gbc/data/development_set_test.txt"
file_paths = open(test_file,'r')
 
n_fft = 1024

features = pd.DataFrame()
result = pd.DataFrame()

gmm_files = [os.path.join(dest,fname) for fname in os.listdir(dest)]
gmm_files = [fname for fname in gmm_files if not '.ipynb' in fname]
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files ]
scaler    = pickle.load(open('scaler','rb'))

# Последовательное чтение аудио-файлов из тестового семпла.
for path in tqdm(file_paths, desc='Score test sample '):
    
    features = pd.DataFrame()
    
    path = path.replace("\\","/").strip()
    speaker = path.split("-",1)[0]
    
    # librosa.load - загрузка аудио-файла.
    sample, sample_rate = librosa.load(source+path)
    
    # Извлечение аудио-признаков.
    features = features.append(pd.Series(audio_features_extraction(sample, sample_rate, n_fft)), ignore_index=True) 

    # Нормирование аудио-признаков.
    features = scaler.transform(features)
    
    # Скоринг каждой моделью
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm = models[i] 
        scores = np.array(gmm.score(features))
        log_likelihood[i] = scores.sum()
        
    # Выбор спикера по наибольшему скору
    winner = np.argmax(log_likelihood)
    result = result.append({'speaker' : speaker,
                           'winner' : gmm_files[winner].split("/",3)[-1]}, ignore_index=True)
    
print('Execution time: ', round((time.time() - start),2))   
print('Точность угадывания, %: ', round(100*result[result['speaker']==result['winner']].shape[0]/result.shape[0],2))

Score test sample : 170it [01:14,  2.27it/s]

Execution time:  74.99

Точность угадывания, %:  98.82


In [ ]:
result.head()